In [ ]:
# train_gpt_memmap_with_perplexity.py
# Full script with: memmap-safe dataset builder + GPT training (RMSNorm + SwiGLU + Attention V=K)
# Adds: validation perplexity calculation + saving of checkpoints, config.json, and metrics.jsonl

import os
import json
import time
import math
from typing import Optional

import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# -----------------------
# Config - edit paths & hyperparams
# -----------------------
DATA_DIR = "/kaggle/input/jklu-en-memap-5gb"   # where wikipedia_tokens.bin & wikipedia_tokens_meta.json live
SINGLEFILE_DIR = "/kaggle/input/jklu-en5gb-singlefile"  # fallback dir for merged_wikipedia.txt
BIN_NAME = "wikipedia_tokens.bin"
META_NAME = "wikipedia_tokens_meta.json"
SINGLEFILE_NAME = "merged_wikipedia.txt"

# Data selection: use only first TRAIN_PERCENT of memmap tokens
TRAIN_PERCENT = 0.70        # <-- use 70% of full memmap tokens
TRAIN_VAL_SPLIT = 0.70      # <-- within the selected tokens, train=70%, val=30%

# Low-memory safe defaults (change if you know your GPU can handle more)
SEQ_LEN = 256        # context length (reduce to save memory)
BATCH_SIZE = 2       # per-step batch size (increase if you have more memory)
GRAD_ACCUM = 1       # accumulate gradients to emulate bigger batch
EPOCHS = 1
LR = 2e-4
NUM_WORKERS = 0      # set to 0 for low-memory, increase if you have spare CPU RAM
PIN_MEMORY = False
SAVE_EVERY_STEPS = 2000
OUTPUT_DIR = "./out_ckpt"
os.makedirs(OUTPUT_DIR, exist_ok=True)

METRICS_LOG = os.path.join(OUTPUT_DIR, "metrics.jsonl")
CONFIG_PATH = os.path.join(OUTPUT_DIR, "config.json")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Chunking when building memmap from text
TOKENIZE_CHUNK_LINES = 1000   # number of lines per chunk to pass to tokenizer

# -----------------------
# Utils: meta & memmap
# -----------------------
def load_meta(meta_path: str) -> dict:
    if not os.path.exists(meta_path):
        return {}
    with open(meta_path, "r", encoding="utf-8") as f:
        return json.load(f)

def infer_dtype_from_meta(meta: dict) -> np.dtype:
    if 'dtype' in meta:
        return np.dtype(meta['dtype'])
    if 'numpy_dtype' in meta:
        return np.dtype(meta['numpy_dtype'])
    return np.int32

def load_memmap_tokens(bin_path: str, meta_path: Optional[str] = None) -> np.memmap:
    meta = load_meta(meta_path) if meta_path and os.path.exists(meta_path) else {}
    dtype = infer_dtype_from_meta(meta)
    if 'num_tokens' in meta:
        length = int(meta['num_tokens'])
    elif 'length' in meta:
        length = int(meta['length'])
    else:
        fsize = os.path.getsize(bin_path)
        length = fsize // dtype.itemsize
    print(f"Opening memmap: {bin_path} (dtype={dtype}, length={length:,})")
    mem = np.memmap(bin_path, dtype=dtype, mode='r', shape=(length,))
    return mem

# -----------------------
# Build memmap from text (chunked, safe)
# -----------------------
def build_memmap_from_text_streaming(text_path: str, out_bin: str, out_meta: str,
                                     tokenizer_name: Optional[str] = None):
    print("🔨 Building memmap from text (streaming chunked tokenizer)...")
    use_hf = False
    tok = None
    if tokenizer_name:
        try:
            from transformers import AutoTokenizer
            tok = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
            use_hf = True
            print(f"Using HF tokenizer: {tokenizer_name}")
        except Exception:
            print("Could not load HF tokenizer — falling back to byte-level chunking.")
            use_hf = False

    if os.path.exists(out_bin):
        print("Removing existing incomplete bin file:", out_bin)
        os.remove(out_bin)

    total_tokens = 0
    dtype = np.int32

    with open(text_path, "r", encoding="utf-8", errors="ignore") as f:
        chunk_lines = []
        for lineno, line in enumerate(f, 1):
            chunk_lines.append(line)
            if lineno % TOKENIZE_CHUNK_LINES == 0:
                txt = "".join(chunk_lines)
                if use_hf:
                    enc = tok(txt, add_special_tokens=False, return_attention_mask=False)
                    ids = np.array(enc["input_ids"], dtype=np.int32)
                else:
                    b = txt.encode("utf-8", errors="ignore")
                    ids = np.frombuffer(b, dtype=np.uint8).astype(np.int32)
                if ids.size:
                    with open(out_bin, "ab") as wf:
                        wf.write(ids.tobytes())
                    total_tokens += ids.size
                chunk_lines = []
                print(f"  tokenized lines up to {lineno:,}  -> tokens total {total_tokens:,}")
        # final small chunk
        if chunk_lines:
            txt = "".join(chunk_lines)
            if use_hf:
                enc = tok(txt, add_special_tokens=False, return_attention_mask=False)
                ids = np.array(enc["input_ids"], dtype=np.int32)
            else:
                b = txt.encode("utf-8", errors="ignore")
                ids = np.frombuffer(b, dtype=np.uint8).astype(np.int32)
            if ids.size:
                with open(out_bin, "ab") as wf:
                    wf.write(ids.tobytes())
                total_tokens += ids.size
            print(f"  final chunk tokenized -> tokens total {total_tokens:,}")

    meta = {
        "dtype": str(dtype),
        "num_tokens": int(total_tokens),
        "tokenizer": tokenizer_name if tokenizer_name else "byte_level_fallback"
    }
    with open(out_meta, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)
    print(f"✅ Wrote memmap bin {out_bin} ({total_tokens:,} tokens) and meta {out_meta}.")

# -----------------------
# Dataset: sample context windows from memmap
# -----------------------
class MemmapTokenDataset(Dataset):
    def __init__(self, memmap: np.memmap, seq_len: int, split: str = "train", split_ratio=0.98):
        assert split in ("train", "val")
        N = memmap.shape[0]
        split_idx = int(N * split_ratio)
        if split == "train":
            self.start = 0
            self.end = split_idx
        else:
            self.start = split_idx
            self.end = N
        self.seq_len = seq_len
        self.memmap = memmap
        self.num_examples = max(0, (self.end - self.start) - seq_len)
    def __len__(self):
        return self.num_examples
    def __getitem__(self, idx):
        pos = self.start + idx
        seq = np.array(self.memmap[pos: pos + self.seq_len + 1], dtype=np.int64)  # +1 for target shift
        input_ids = torch.from_numpy(seq[:-1]).long()
        target_ids = torch.from_numpy(seq[1:]).long()
        return input_ids, target_ids

def collate_batch(batch):
    inputs = torch.stack([b[0] for b in batch], dim=0)
    targets = torch.stack([b[1] for b in batch], dim=0)
    return inputs, targets

# -----------------------
# Model components: RMSNorm, SwiGLU, Attention (V=K)
# -----------------------
class RMSNormSimple(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-8):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))
    def forward(self, x):
        rms = x.pow(2).mean(-1, keepdim=True).add(self.eps).sqrt()
        return x / rms * self.weight

class SwiGLUFFN_simple(nn.Module):
    def __init__(self, d_model: int, expansion: int = 4, dropout: float = 0.1):
        super().__init__()
        hidden = d_model * expansion
        self.project_in = nn.Linear(d_model, hidden * 2)
        self.project_out = nn.Linear(hidden, d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x_proj = self.project_in(x)
        a, b = x_proj.chunk(2, dim=-1)
        x_ff = a * torch.nn.functional.silu(b)
        x_out = self.project_out(x_ff)
        return self.dropout(x_out)

class CausalKVAttention_simple(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dropout: float = 0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.scale = (self.head_dim) ** -0.5
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)
        self.attn_dropout = nn.Dropout(dropout)
        self.proj_dropout = nn.Dropout(dropout)
    def _shape(self, x):
        B, T, D = x.shape
        return x.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
    def _unshape(self, x):
        B, nh, T, hd = x.shape
        return x.transpose(1, 2).contiguous().view(B, T, nh * hd)
    def forward(self, x, cache_k: Optional[torch.Tensor] = None):
        B, T, D = x.shape
        q = self.q_proj(x)
        k = self.k_proj(x)
        qh = self._shape(q)
        kh = self._shape(k)
        vh = kh  # V = K
        if cache_k is not None:
            kh = torch.cat([cache_k, kh], dim=2)
            vh = kh
        T_total = kh.size(2)
        scores = torch.matmul(qh, kh.transpose(-2, -1)) * self.scale
        device = x.device
        causal_mask = torch.triu(torch.ones((T, T_total), dtype=torch.bool, device=device), diagonal=1)
        scores = scores.masked_fill(causal_mask.unsqueeze(0).unsqueeze(0), float('-inf'))
        attn = torch.nn.functional.softmax(scores, dim=-1)
        attn = self.attn_dropout(attn)
        ctx = torch.matmul(attn, vh)
        ctx = self._unshape(ctx)
        out = self.out_proj(ctx)
        out = self.proj_dropout(out)
        new_cache_k = kh if cache_k is not None else None
        return out, new_cache_k

class GPTBlockKVasK_simple(nn.Module):
    def __init__(self, d_model: int, n_heads: int, ffn_expansion: int = 4,
                 attn_dropout: float = 0.1, ffn_dropout: float = 0.1):
        super().__init__()
        self.rms1 = RMSNormSimple(d_model)
        self.attn = CausalKVAttention_simple(d_model, n_heads, dropout=attn_dropout)
        self.rdrop1 = nn.Dropout(attn_dropout)
        self.rms2 = RMSNormSimple(d_model)
        self.ffn = SwiGLUFFN_simple(d_model, expansion=ffn_expansion, dropout=ffn_dropout)
        self.rdrop2 = nn.Dropout(ffn_dropout)
    def forward(self, x, cache_k: Optional[torch.Tensor] = None):
        x_norm = self.rms1(x)
        attn_out, new_cache_k = self.attn(x_norm, cache_k=cache_k)
        x = x + self.rdrop1(attn_out)
        x_norm = self.rms2(x)
        ffn_out = self.ffn(x_norm)
        x = x + self.rdrop2(ffn_out)
        return x, new_cache_k

class MiniGPT(nn.Module):
    def __init__(self, vocab_size: int, d_model: int = 512, n_heads: int = 8, n_layers: int = 6,
                 seq_len: int = 256, ffn_expansion: int = 4, dropout: float = 0.1):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Parameter(torch.zeros(1, seq_len, d_model))
        self.blocks = nn.ModuleList([GPTBlockKVasK_simple(d_model, n_heads, ffn_expansion, dropout, dropout)
                                     for _ in range(n_layers)])
        self.ln_f = RMSNormSimple(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        self.seq_len = seq_len
    def forward(self, input_ids, cache_k: Optional[torch.Tensor] = None):
        B, T = input_ids.shape
        x = self.tok_emb(input_ids)
        x = x + self.pos_emb[:, :T, :]
        new_cache = None
        for blk in self.blocks:
            x, new_cache = blk(x, cache_k=cache_k)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits, new_cache

# -----------------------
# Helpers: save checkpoint & append metrics
# -----------------------
def save_checkpoint(path: str, model: nn.Module, optimizer: torch.optim.Optimizer, global_step: int, extra: dict):
    payload = {
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "global_step": global_step,
        "extra": extra
    }
    torch.save(payload, path)
    # also save config.json with extra (overwrites but that's fine)
    config = extra.get("config", {})
    with open(CONFIG_PATH, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2)

def append_metrics(epoch: int, global_step: int, val_loss: float, perplexity: float):
    rec = {
        "timestamp": time.time(),
        "epoch": epoch,
        "global_step": global_step,
        "val_loss": float(val_loss),
        "perplexity": float(perplexity)
    }
    with open(METRICS_LOG, "a", encoding="utf-8") as f:
        f.write(json.dumps(rec) + "\n")

# -----------------------
# Training function: memmap prints + safe loop with AMP & OOM handling
# -----------------------
def train():
    print("\n==============================")
    print("📂 Checking dataset & memmap files")
    print("==============================\n")

    bin_path = os.path.join(DATA_DIR, BIN_NAME)
    meta_path = os.path.join(DATA_DIR, META_NAME)
    text_path = os.path.join(SINGLEFILE_DIR, SINGLEFILE_NAME)

    print(f"Looking for bin: {bin_path}")
    print(f"Looking for meta: {meta_path}")

    if not os.path.exists(bin_path):
        print("❌ Memmap .bin not found.")
        if os.path.exists(text_path):
            print(f"Found fallback text: {text_path}")
            build_memmap_from_text_streaming(text_path, bin_path, meta_path, tokenizer_name=None)
        else:
            raise FileNotFoundError(f"Neither {bin_path} nor {text_path} found. Provide dataset.")

    # load memmap
    mem = load_memmap_tokens(bin_path, meta_path)
    total_tokens = len(mem)
    print(f"✔️ Loaded memmap: shape={mem.shape}, dtype={mem.dtype}, total tokens={total_tokens:,}")

    # -------------------------------
    # Limit dataset to TRAIN_PERCENT
    # -------------------------------
    MAX_TOKENS = int(total_tokens * TRAIN_PERCENT)
    print(f"📏 Using only first {TRAIN_PERCENT*100:.0f}% of data → {MAX_TOKENS:,} tokens")
    mem = mem[:MAX_TOKENS]

    meta = load_meta(meta_path) if os.path.exists(meta_path) else {}
    vocab_size = meta.get("vocab_size", None)
    if vocab_size is None:
        try:
            vocab_size = int(np.max(mem)) + 1
            print(f"Inferred vocab_size: {vocab_size}")
        except Exception:
            vocab_size = 65536
            print(f"Could not infer vocab_size. Defaulting to {vocab_size}")

    print("\n📦 Creating datasets (on sliced memmap)...")
    train_ds = MemmapTokenDataset(mem, seq_len=SEQ_LEN, split="train", split_ratio=TRAIN_VAL_SPLIT)
    val_ds = MemmapTokenDataset(mem, seq_len=SEQ_LEN, split="val", split_ratio=TRAIN_VAL_SPLIT)
    print(f"✔️ Train samples: {len(train_ds):,}")
    print(f"✔️ Val samples  : {len(val_ds):,}")

    print("\n📦 Building DataLoaders...")
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=collate_batch, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                            collate_fn=collate_batch, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
    print("DataLoaders ready.")

    print("\n🧠 Initializing model...")
    model = MiniGPT(vocab_size=vocab_size, d_model=512, n_heads=8, n_layers=6, seq_len=SEQ_LEN).to(DEVICE)
    print("Model on device:", DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

    # write config file (initial)
    config = {
        "DATA_DIR": DATA_DIR,
        "BIN_NAME": BIN_NAME,
        "META_NAME": META_NAME,
        "TRAIN_PERCENT": TRAIN_PERCENT,
        "TRAIN_VAL_SPLIT": TRAIN_VAL_SPLIT,
        "SEQ_LEN": SEQ_LEN,
        "BATCH_SIZE": BATCH_SIZE,
        "GRAD_ACCUM": GRAD_ACCUM,
        "EPOCHS": EPOCHS,
        "LR": LR,
        "NUM_WORKERS": NUM_WORKERS,
        "DEVICE": DEVICE,
        "vocab_size": int(vocab_size)
    }
    with open(CONFIG_PATH, "w", encoding="utf-8") as f:
        json.dump(config, f, indent=2)

    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE == "cuda"))
    global_step = 0
    model.train()

    for epoch in range(EPOCHS):
        print("\n=====================================")
        print(f"🚀 Starting Epoch {epoch+1}/{EPOCHS}")
        print("=====================================\n")
        epoch_start = time.time()
        steps_per_epoch = len(train_loader)
        print(f"Steps this epoch: {steps_per_epoch}, BATCH_SIZE: {BATCH_SIZE}, SEQ_LEN: {SEQ_LEN}")

        for batch_idx, (input_ids, target_ids) in enumerate(train_loader):
            # move to device
            input_ids = input_ids.to(DEVICE, non_blocking=True)
            target_ids = target_ids.to(DEVICE, non_blocking=True)

            # forward / backward with AMP
            try:
                with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
                    logits, _ = model(input_ids)
                    loss = criterion(logits.view(-1, logits.size(-1)), target_ids.view(-1))
                    loss = loss / GRAD_ACCUM

                scaler.scale(loss).backward()

                if (batch_idx + 1) % GRAD_ACCUM == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                    global_step += 1

                # periodic prints
                if global_step % 50 == 0:
                    print(f"[Epoch {epoch+1}] Global step {global_step} | batch {batch_idx+1}/{steps_per_epoch} | loss {loss.item()*GRAD_ACCUM:.6f}")

            except RuntimeError as e:
                if 'out of memory' in str(e).lower():
                    print(f"⚠️ OOM at batch {batch_idx+1} — clearing grad/cache and skipping step.")
                    optimizer.zero_grad()
                    if DEVICE == "cuda":
                        torch.cuda.empty_cache()
                    continue
                else:
                    raise

            # ETA every 200 batch reads
            if (batch_idx + 1) % 200 == 0:
                elapsed = time.time() - epoch_start
                steps_done = batch_idx + 1
                avg_step = elapsed / max(1, steps_done)
                eta = (steps_per_epoch - steps_done) * avg_step
                print(f"⏳ ETA for epoch: {eta/60:.2f} minutes")

            # checkpointing
            if global_step and global_step % SAVE_EVERY_STEPS == 0:
                ckpt_path = os.path.join(OUTPUT_DIR, f"ckpt_step{global_step}.pt")
                save_checkpoint(ckpt_path, model, optimizer, global_step, {"config": config})
                print("💾 Saved checkpoint ->", ckpt_path)
                if DEVICE == "cuda":
                    torch.cuda.empty_cache()

        epoch_time = time.time() - epoch_start
        print(f"\n🏁 Epoch {epoch+1} finished in {epoch_time/60:.2f} minutes")

        # validation (small sample) — compute average loss and perplexity
        print("\n🔍 Running quick validation sample...")
        model.eval()
        with torch.no_grad():
            val_losses = []
            for i, (input_ids, target_ids) in enumerate(val_loader):
                input_ids = input_ids.to(DEVICE, non_blocking=True)
                target_ids = target_ids.to(DEVICE, non_blocking=True)
                logits, _ = model(input_ids)
                loss = criterion(logits.view(-1, logits.size(-1)), target_ids.view(-1))
                val_losses.append(loss.item())
                if i >= 50:  # sample up to 51 batches for a reasonable estimate (adjustable)
                    break
            if val_losses:
                avg_val_loss = float(sum(val_losses) / len(val_losses))
                # perplexity = exp(avg_val_loss)
                try:
                    perp = float(math.exp(avg_val_loss))
                    # protect against overflow
                    if math.isinf(perp) or perp > 1e30:
                        perp = float('inf')
                except OverflowError:
                    perp = float('inf')
                print(f"✔️ Validation loss (sample of {len(val_losses)} batches): {avg_val_loss:.5f}")
                print(f"✔️ Perplexity (sample): {perp:.3f}")
                # append metrics to log file
                append_metrics(epoch + 1, global_step, avg_val_loss, perp)
                # save checkpoint for this epoch (with metrics)
                ckpt_path = os.path.join(OUTPUT_DIR, f"ckpt_epoch{epoch+1}_step{global_step}.pt")
                extra = {"config": config, "val_loss": avg_val_loss, "perplexity": perp, "epoch": epoch + 1}
                save_checkpoint(ckpt_path, model, optimizer, global_step, extra)
                print("💾 Saved epoch checkpoint ->", ckpt_path)
        model.train()

    # final save + config and metrics
    final_path = os.path.join(OUTPUT_DIR, "final_ckpt.pt")
    save_checkpoint(final_path, model, optimizer, global_step, {"config": config})
    print("Training complete. Saved final checkpoint ->", final_path)
    print("Config saved to:", CONFIG_PATH)
    print("Metrics log appended to:", METRICS_LOG)

if __name__ == "__main__":
    train()


In [4]:
# test_gpt_inference.py
# Load trained checkpoint and generate text from prompts

import os
import json
import torch
from torch import nn
import numpy as np
from typing import Optional

# -----------------------
# Config - edit paths
# -----------------------
CHECKPOINT_PATH = "/kaggle/input/46k-itr/ckpt_step46000.pt"  # or any specific checkpoint
CONFIG_PATH = "/kaggle/input/confic/confic.json"
META_PATH = "/kaggle/input/jklu-en-memap-5gb/wikipedia_tokens_meta.json"  # for vocab info

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_NEW_TOKENS = 100  # how many tokens to generate
TEMPERATURE = 0.8     # higher = more random, lower = more deterministic
TOP_K = 50            # only sample from top-k tokens (0 = disabled)

# -----------------------
# Model components (same as training)
# -----------------------
class RMSNormSimple(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-8):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))
    def forward(self, x):
        rms = x.pow(2).mean(-1, keepdim=True).add(self.eps).sqrt()
        return x / rms * self.weight

class SwiGLUFFN_simple(nn.Module):
    def __init__(self, d_model: int, expansion: int = 4, dropout: float = 0.1):
        super().__init__()
        hidden = d_model * expansion
        self.project_in = nn.Linear(d_model, hidden * 2)
        self.project_out = nn.Linear(hidden, d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x_proj = self.project_in(x)
        a, b = x_proj.chunk(2, dim=-1)
        x_ff = a * torch.nn.functional.silu(b)
        x_out = self.project_out(x_ff)
        return self.dropout(x_out)

class CausalKVAttention_simple(nn.Module):
    def __init__(self, d_model: int, n_heads: int, dropout: float = 0.1):
        super().__init__()
        assert d_model % n_heads == 0
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.scale = (self.head_dim) ** -0.5
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)
        self.attn_dropout = nn.Dropout(dropout)
        self.proj_dropout = nn.Dropout(dropout)
    def _shape(self, x):
        B, T, D = x.shape
        return x.view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
    def _unshape(self, x):
        B, nh, T, hd = x.shape
        return x.transpose(1, 2).contiguous().view(B, T, nh * hd)
    def forward(self, x, cache_k: Optional[torch.Tensor] = None):
        B, T, D = x.shape
        q = self.q_proj(x)
        k = self.k_proj(x)
        qh = self._shape(q)
        kh = self._shape(k)
        vh = kh  # V = K
        if cache_k is not None:
            kh = torch.cat([cache_k, kh], dim=2)
            vh = kh
        T_total = kh.size(2)
        scores = torch.matmul(qh, kh.transpose(-2, -1)) * self.scale
        device = x.device
        causal_mask = torch.triu(torch.ones((T, T_total), dtype=torch.bool, device=device), diagonal=1)
        scores = scores.masked_fill(causal_mask.unsqueeze(0).unsqueeze(0), float('-inf'))
        attn = torch.nn.functional.softmax(scores, dim=-1)
        attn = self.attn_dropout(attn)
        ctx = torch.matmul(attn, vh)
        ctx = self._unshape(ctx)
        out = self.out_proj(ctx)
        out = self.proj_dropout(out)
        new_cache_k = kh if cache_k is not None else None
        return out, new_cache_k

class GPTBlockKVasK_simple(nn.Module):
    def __init__(self, d_model: int, n_heads: int, ffn_expansion: int = 4,
                 attn_dropout: float = 0.1, ffn_dropout: float = 0.1):
        super().__init__()
        self.rms1 = RMSNormSimple(d_model)
        self.attn = CausalKVAttention_simple(d_model, n_heads, dropout=attn_dropout)
        self.rdrop1 = nn.Dropout(attn_dropout)
        self.rms2 = RMSNormSimple(d_model)
        self.ffn = SwiGLUFFN_simple(d_model, expansion=ffn_expansion, dropout=ffn_dropout)
        self.rdrop2 = nn.Dropout(ffn_dropout)
    def forward(self, x, cache_k: Optional[torch.Tensor] = None):
        x_norm = self.rms1(x)
        attn_out, new_cache_k = self.attn(x_norm, cache_k=cache_k)
        x = x + self.rdrop1(attn_out)
        x_norm = self.rms2(x)
        ffn_out = self.ffn(x_norm)
        x = x + self.rdrop2(ffn_out)
        return x, new_cache_k

class MiniGPT(nn.Module):
    def __init__(self, vocab_size: int, d_model: int = 512, n_heads: int = 8, n_layers: int = 6,
                 seq_len: int = 256, ffn_expansion: int = 4, dropout: float = 0.1):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Parameter(torch.zeros(1, seq_len, d_model))
        self.blocks = nn.ModuleList([GPTBlockKVasK_simple(d_model, n_heads, ffn_expansion, dropout, dropout)
                                     for _ in range(n_layers)])
        self.ln_f = RMSNormSimple(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        self.seq_len = seq_len
    def forward(self, input_ids, cache_k: Optional[torch.Tensor] = None):
        B, T = input_ids.shape
        x = self.tok_emb(input_ids)
        x = x + self.pos_emb[:, :T, :]
        new_cache = None
        for blk in self.blocks:
            x, new_cache = blk(x, cache_k=cache_k)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits, new_cache

# -----------------------
# Load checkpoint
# -----------------------
def load_checkpoint(ckpt_path: str, device: str = "cpu"):
    print(f"Loading checkpoint from: {ckpt_path}")
    checkpoint = torch.load(ckpt_path, map_location=device)
    return checkpoint

# -----------------------
# Tokenizer support: tiktoken, HuggingFace, or byte-level
# -----------------------
_tokenizer_cache = None

def get_tokenizer(tokenizer_type: str):
    """Get or create tokenizer (cached)"""
    global _tokenizer_cache
    if _tokenizer_cache is not None:
        return _tokenizer_cache
    
    if tokenizer_type == "byte_level":
        _tokenizer_cache = None
        return None
    
    # Try tiktoken first (for GPT-2, GPT-3 style tokenizers)
    if "tiktoken" in tokenizer_type.lower() or "gpt" in tokenizer_type.lower():
        try:
            import tiktoken
            # tiktoken_gpt2 maps to gpt2 encoding
            encoding_name = "gpt2"  # or "cl100k_base" for GPT-3.5/4
            enc = tiktoken.get_encoding(encoding_name)
            _tokenizer_cache = ("tiktoken", enc)
            print(f"✅ Loaded tiktoken encoder: {encoding_name}")
            return _tokenizer_cache
        except ImportError:
            print("⚠️ tiktoken not installed. Install with: pip install tiktoken")
        except Exception as e:
            print(f"⚠️ Could not load tiktoken: {e}")
    
    # Try HuggingFace tokenizer
    try:
        from transformers import AutoTokenizer
        tok = AutoTokenizer.from_pretrained(tokenizer_type, use_fast=True)
        _tokenizer_cache = ("hf", tok)
        print(f"✅ Loaded HF tokenizer: {tokenizer_type}")
        return _tokenizer_cache
    except Exception as e:
        print(f"⚠️ Could not load HF tokenizer: {e}")
    
    _tokenizer_cache = None
    return None

def encode_text(text: str, tokenizer_type: str = "byte_level") -> list:
    """Convert text to token IDs"""
    if tokenizer_type == "byte_level":
        return list(text.encode("utf-8", errors="ignore"))
    
    tok = get_tokenizer(tokenizer_type)
    if tok is None:
        print("⚠️ Falling back to byte-level encoding")
        return list(text.encode("utf-8", errors="ignore"))
    
    tok_type, tok_obj = tok
    if tok_type == "tiktoken":
        return tok_obj.encode(text)
    elif tok_type == "hf":
        return tok_obj.encode(text, add_special_tokens=False)
    else:
        return list(text.encode("utf-8", errors="ignore"))

def decode_tokens(tokens: list, tokenizer_type: str = "byte_level") -> str:
    """Convert token IDs back to text"""
    if tokenizer_type == "byte_level":
        try:
            return bytes(tokens).decode("utf-8", errors="ignore")
        except:
            return ""
    
    tok = get_tokenizer(tokenizer_type)
    if tok is None:
        try:
            return bytes(tokens).decode("utf-8", errors="ignore")
        except:
            return ""
    
    tok_type, tok_obj = tok
    if tok_type == "tiktoken":
        return tok_obj.decode(tokens)
    elif tok_type == "hf":
        return tok_obj.decode(tokens)
    else:
        try:
            return bytes(tokens).decode("utf-8", errors="ignore")
        except:
            return ""

# -----------------------
# Generation with temperature and top-k sampling
# -----------------------
@torch.no_grad()
def generate(model, prompt_tokens: list, max_new_tokens: int = 100, 
             temperature: float = 1.0, top_k: int = 0, device: str = "cpu"):
    """Generate text autoregressively from prompt"""
    model.eval()
    
    # Convert prompt to tensor
    input_ids = torch.tensor([prompt_tokens], dtype=torch.long).to(device)
    generated = prompt_tokens.copy()
    
    for _ in range(max_new_tokens):
        # Truncate if exceeds seq_len
        if input_ids.size(1) > model.seq_len:
            input_ids = input_ids[:, -model.seq_len:]
        
        # Forward pass
        logits, _ = model(input_ids)
        
        # Get logits for last token
        logits = logits[:, -1, :] / temperature
        
        # Apply top-k filtering
        if top_k > 0:
            indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
            logits[indices_to_remove] = float('-inf')
        
        # Sample from distribution
        probs = torch.nn.functional.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        # Append to sequence
        generated.append(next_token.item())
        input_ids = torch.cat([input_ids, next_token], dim=1)
        
        # Optional: stop at special tokens (e.g., 0, 1, 2 for padding/eos)
        # if next_token.item() in [0, 1, 2]:
        #     break
    
    return generated

# -----------------------
# Main inference function
# -----------------------
def main():
    print("\n" + "="*50)
    print("🧪 GPT Model Inference Script")
    print("="*50 + "\n")
    
    # Load config
    if os.path.exists(CONFIG_PATH):
        print(f"Loading config from: {CONFIG_PATH}")
        with open(CONFIG_PATH, "r") as f:
            config = json.load(f)
        vocab_size = config.get("vocab_size", 256)
        seq_len = config.get("SEQ_LEN", 256)
        print(f"Config loaded: vocab_size={vocab_size}, seq_len={seq_len}")
    else:
        print("⚠️ Config not found, using defaults")
        vocab_size = 256  # byte-level default
        seq_len = 256
    
    # Check if using HF tokenizer
    tokenizer_type = "byte_level"
    if os.path.exists(META_PATH):
        with open(META_PATH, "r") as f:
            meta = json.load(f)
            tokenizer_type = meta.get("tokenizer", "byte_level")
    
    print(f"Tokenizer type: {tokenizer_type}\n")
    
    # Initialize model
    print("Initializing model...")
    model = MiniGPT(
        vocab_size=vocab_size,
        d_model=512,
        n_heads=8,
        n_layers=6,
        seq_len=seq_len,
        ffn_expansion=4,
        dropout=0.1
    ).to(DEVICE)
    
    # Load checkpoint
    if not os.path.exists(CHECKPOINT_PATH):
        print(f"❌ Checkpoint not found: {CHECKPOINT_PATH}")
        return
    
    checkpoint = load_checkpoint(CHECKPOINT_PATH, DEVICE)
    model.load_state_dict(checkpoint["model_state"])
    model.eval()
    
    print(f"✅ Model loaded successfully!")
    print(f"Global step: {checkpoint.get('global_step', 'N/A')}")
    if 'extra' in checkpoint and 'val_loss' in checkpoint['extra']:
        print(f"Val loss: {checkpoint['extra']['val_loss']:.4f}")
        print(f"Perplexity: {checkpoint['extra'].get('perplexity', 'N/A')}")
    print()
    
    # Interactive loop
    print("="*50)
    print("Ready for inference! Type 'quit' to exit.")
    print("="*50 + "\n")
    
    while True:
        prompt = input("Enter prompt: ").strip()
        
        if prompt.lower() in ['quit', 'exit', 'q']:
            print("Exiting...")
            break
        
        if not prompt:
            continue
        
        # Encode prompt
        prompt_tokens = encode_text(prompt, tokenizer_type)
        print(f"\nPrompt tokens ({len(prompt_tokens)}): {prompt_tokens[:20]}...")
        
        # Generate
        print(f"Generating {MAX_NEW_TOKENS} tokens...")
        generated_tokens = generate(
            model,
            prompt_tokens,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=TEMPERATURE,
            top_k=TOP_K,
            device=DEVICE
        )
        
        # Decode
        generated_text = decode_tokens(generated_tokens, tokenizer_type)
        
        print("\n" + "-"*50)
        print("Generated text:")
        print("-"*50)
        print(generated_text)
        print("-"*50 + "\n")

if __name__ == "__main__":
    main()


🧪 GPT Model Inference Script

Loading config from: /kaggle/input/confic/confic.json
Config loaded: vocab_size=50257, seq_len=256
Tokenizer type: tiktoken_gpt2

Initializing model...
Loading checkpoint from: /kaggle/input/46k-itr/ckpt_step46000.pt
✅ Model loaded successfully!
Global step: 46000

Ready for inference! Type 'quit' to exit.



Enter prompt:  The capital of India is 


✅ Loaded tiktoken encoder: gpt2

Prompt tokens (5): [464, 3139, 286, 3794, 318]...
Generating 100 tokens...

--------------------------------------------------
Generated text:
--------------------------------------------------
The capital of India is a popular country, the most popular sport. the rugby is the highest rugby world cup in the british league. the club is the main club (fucnfaa) and the scottish competition in the united states, and has a number of competitions in the united states, including in the united states: the club's premier league, the team is represented in the "london cup". the club is a team is a major league which is the british club (ro
--------------------------------------------------



Enter prompt:  America has the 



Prompt tokens (3): [18165, 468, 262]...
Generating 100 tokens...

--------------------------------------------------
Generated text:
--------------------------------------------------
America has the largest producer in the country, especially in the nation. in early april 2007, the company adopted a new "new american" that ran all of its own label, which included the last name of the band's "the band" on september 16, 2008.

on june 7, 2015, a live album, "the new orleans" was released at the same time. it was certified $3 million in 2009.

on november 7, 2017
--------------------------------------------------



...


Prompt tokens (4): [83, 1045, 2369, 364]...
Generating 100 tokens...

--------------------------------------------------
Generated text:
--------------------------------------------------
tata moters, he left for the first time in the second round of the second time.

he played the celtics, the youngest evertonio in the first round of the wolverineken cup but was the longest in the summer olympics in the early 1920s. he was a member of the national team that year in the league with their first-round knockout in the league. although the club won the cec cup, he was the scottish cup the second team to
--------------------------------------------------



Enter prompt:  JK lakhsmipat university is 



Prompt tokens (8): [41, 42, 29002, 5796, 541, 265, 6403, 318]...
Generating 100 tokens...

--------------------------------------------------
Generated text:
--------------------------------------------------
JK lakhsmipat university is called "the "b.p. r. n. s. t. k. k. h. m. b. s. l. f. l. d. g. in 1976, the school was the official language of the u.s. government (formerly u. u.s. highway 8 at the end of december 1981) to develop the same name (the university of caledonian), established in 1993 at the school building on the site of the district and the
--------------------------------------------------



Enter prompt:  exit


Exiting...


In [1]:
# # generate_from_ckpt.py
# # Usage:
# #  - Edit CHECKPOINT_PATH and optionally TOKENIZER_NAME
# #  - Run: python generate_from_ckpt.py
# # Produces: printed generated text for the prompt below.

# import os
# import json
# import math
# import torch
# import torch.nn.functional as F
# from typing import List, Optional

# # --- Edit these ---
# CHECKPOINT_PATH = "/kaggle/input/checkpoint/ckpt_step8000.pt"   # a checkpoint saved by your train script
# TOKENIZER_NAME = None   # e.g. "gpt2" if you used HF tokenizer; None to use byte-level fallback
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# # ------------------

# # If your MiniGPT and component classes are in a module, import them.
# # If not, paste the MiniGPT and component class definitions here (or run from the same file).
# # For convenience, we'll import from train_gpt_memmap if it's in the same folder.
# try:
#     from train_gpt_memmap import MiniGPT, load_meta, load_memmap_tokens  # type: ignore
# except Exception:
#     # If import fails, assume MiniGPT class is in the same file or user will paste it.
#     MiniGPT = None

# # -----------------------
# # Tokenizer helpers
# # -----------------------
# def load_tokenizer(tokenizer_name: Optional[str]):
#     if tokenizer_name:
#         try:
#             from transformers import AutoTokenizer
#             tok = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)
#             print("Loaded HF tokenizer:", tokenizer_name)
#             return tok
#         except Exception as e:
#             print("Failed to load HF tokenizer:", e)
#             print("Falling back to byte-level tokenizer.")
#     return None

# def encode_text(tok, text: str) -> List[int]:
#     if tok is not None:
#         return tok(text, add_special_tokens=False)["input_ids"]
#     else:
#         b = text.encode("utf-8", errors="ignore")
#         return list(b)  # bytes -> ints 0..255

# def decode_tokens(tok, ids: List[int]) -> str:
#     if tok is not None:
#         return tok.decode(ids) if hasattr(tok, "decode") else tok.convert_tokens_to_string(tok.convert_ids_to_tokens(ids))
#     else:
#         # byte-level decode
#         return bytes([i % 256 for i in ids]).decode("utf-8", errors="ignore")

# # -----------------------
# # Sampling utilities
# # -----------------------
# def top_k_top_p_filtering(logits, top_k=0, top_p=0.0):
#     """ Filter a distribution of logits using top-k and/or nucleus (top-p). """
#     top_k = int(top_k)
#     if top_k > 0:
#         values, _ = torch.topk(logits, top_k)
#         min_values = values[:, -1].unsqueeze(1)
#         logits = torch.where(logits < min_values, torch.full_like(logits, -float("Inf")), logits)
#     if top_p > 0.0:
#         sorted_logits, sorted_indices = torch.sort(logits, descending=True)
#         cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
#         # remove tokens with cumulative probability above top_p
#         sorted_mask = cumulative_probs > top_p
#         # shift the mask to keep first token above threshold
#         sorted_mask[..., 1:] = sorted_mask[..., :-1].clone()
#         sorted_mask[..., 0] = False
#         indices_to_remove = sorted_mask.scatter(1, sorted_indices, sorted_mask)
#         logits = logits.masked_fill(indices_to_remove, -float("Inf"))
#     return logits

# # -----------------------
# # Simple generator (no KV caching)
# # -----------------------
# def generate_simple(model, tokenizer, prompt: str, max_new_tokens=64, temperature=1.0,
#                     top_k=50, top_p=0.0, do_sample=True):
#     model.eval()
#     tok = load_tokenizer(TOKENIZER_NAME) if tokenizer is None else tokenizer
#     ids = encode_text(tok, prompt)
#     input_ids = torch.tensor([ids], dtype=torch.long, device=DEVICE)
#     generated = ids.copy()

#     for step in range(max_new_tokens):
#         with torch.no_grad():
#             logits, _ = model(input_ids)  # (B, T, V)
#             logits = logits[:, -1, :] / max(1e-8, temperature)  # last token logits
#             logits = top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p)
#             probs = F.softmax(logits, dim=-1)

#             if do_sample:
#                 next_token = torch.multinomial(probs, num_samples=1).squeeze(1).cpu().item()
#             else:
#                 next_token = torch.argmax(probs, dim=-1).cpu().item()

#         generated.append(int(next_token))
#         # feed the newly generated token for next step
#         input_ids = torch.tensor([generated], dtype=torch.long, device=DEVICE)

#     text = decode_tokens(tok, generated)
#     return text, generated

# # -----------------------
# # Faster generator with per-layer KV cache
# # (Works only if model.blocks return per-layer cache objects — we adapt to the MiniGPT used earlier)
# # -----------------------
# def generate_with_cache(model, tokenizer, prompt: str, max_new_tokens=64, temperature=1.0,
#                         top_k=50, top_p=0.0, do_sample=True):
#     """
#     This implementation keeps per-block cached keys (kh) and reuses them.
#     It assumes model.blocks is a ModuleList and that each block's attention supports
#     passing a cache_k argument and returning new_cache_k for that block.
#     The train script's CausalKVAttention_simple returns per-block kh as `new_cache_k`
#     but MiniGPT as provided returned only final cache; to use this function you must
#     adapt MiniGPT.forward to return a list of caches (one per block) or call blocks manually.
#     Below we call blocks manually to gather caches.
#     """
#     model.eval()
#     tok = load_tokenizer(TOKENIZER_NAME) if tokenizer is None else tokenizer
#     ids = encode_text(tok, prompt)
#     input_ids = torch.tensor([ids], dtype=torch.long, device=DEVICE)
#     generated = ids.copy()

#     # initialize per-layer cache to None
#     layer_caches = [None] * len(model.blocks)  # each entry shape: (B, n_heads, T_cache, head_dim)

#     # first pass: feed the prompt all at once to fill the caches
#     # We'll run block-by-block so we can collect caches per-block.
#     with torch.no_grad():
#         x = model.tok_emb(input_ids) + model.pos_emb[:, : input_ids.size(1), :].to(DEVICE)
#         new_caches = []
#         for i, blk in enumerate(model.blocks):
#             # call block with cache_k (None for prompt)
#             # We expect block to return (x, new_cache_k)
#             x, new_cache_k = blk(x, cache_k=layer_caches[i])
#             new_caches.append(new_cache_k)
#         layer_caches = new_caches
#         x = model.ln_f(x)
#         logits = model.lm_head(x)
#     # generation loop (one token at a time)
#     for step in range(max_new_tokens):
#         # prepare input embedding for last token only
#         last_token_id = torch.tensor([[generated[-1]]], dtype=torch.long, device=DEVICE)
#         x = model.tok_emb(last_token_id)  # (1,1,d)
#         # add positional embedding for next position
#         pos_idx = len(generated)  # zero-based position to index pos_emb
#         x = x + model.pos_emb[:, pos_idx : pos_idx + 1, :].to(DEVICE)

#         # pass through blocks reusing caches (append new keys)
#         new_caches = []
#         for i, blk in enumerate(model.blocks):
#             x, new_cache_k = blk(x, cache_k=layer_caches[i])
#             new_caches.append(new_cache_k)
#         layer_caches = new_caches

#         x = model.ln_f(x)
#         logits = model.lm_head(x)  # (1,1,V)
#         logits = logits[:, -1, :] / max(1e-8, temperature)
#         logits = top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p)
#         probs = F.softmax(logits, dim=-1)

#         if do_sample:
#             next_token = torch.multinomial(probs, num_samples=1).squeeze(1).cpu().item()
#         else:
#             next_token = torch.argmax(probs, dim=-1).cpu().item()

#         generated.append(int(next_token))

#     text = decode_tokens(tok, generated)
#     return text, generated

# # -----------------------
# # Load model checkpoint
# # -----------------------
# def load_model_from_checkpoint(path: str):
#     ckpt = torch.load(path, map_location="cpu")
#     # ckpt payload might be {"model_state": ..., ...} or a raw state_dict
#     if "model_state" in ckpt:
#         state = ckpt["model_state"]
#     else:
#         state = ckpt
#     # load model config (if saved) to construct the MiniGPT correctly
#     # try to infer vocab_size and seq_len from ckpt or config file
#     config = {}
#     config_path = os.path.join(os.path.dirname(path), "config.json")
#     if os.path.exists(config_path):
#         with open(config_path, "r", encoding="utf-8") as f:
#             config = json.load(f)
#     vocab_size = config.get("vocab_size", 65536)
#     seq_len = config.get("SEQ_LEN", 256)
#     # instantiate model (match your training model settings)
#     if MiniGPT is None:
#         raise RuntimeError("MiniGPT class not available. Import it or copy the class into this file.")
#     model = MiniGPT(vocab_size=vocab_size, d_model=512, n_heads=8, n_layers=6, seq_len=seq_len)
#     model.load_state_dict(state, strict=False)
#     model.to(DEVICE)
#     model.eval()
#     return model

# # -----------------------
# # Example usage
# # -----------------------
# if __name__ == "__main__":
#     # load tokenizer & model
#     tokenizer = load_tokenizer(TOKENIZER_NAME)
#     model = load_model_from_checkpoint(CHECKPOINT_PATH)

#     # choose a prompt
#     prompt = "In a future where models write code, a developer asked:"
#     print("Prompt:", prompt)
#     print("Generating with simple (no-cache) generator...")
#     text, ids = generate_simple(model, tokenizer, prompt, max_new_tokens=64, temperature=0.8, top_k=40, top_p=0.9, do_sample=True)
#     print("\n=== GENERATED (simple) ===\n")
#     print(text)
#     print("\nToken ids:", ids[-40:])

#     # if you want faster generation and your model supports per-block caching as above:
#     try:
#         print("\nGenerating with cache-enabled generator (faster)...")
#         text2, ids2 = generate_with_cache(model, tokenizer, prompt, max_new_tokens=64, temperature=0.8, top_k=40, top_p=0.9, do_sample=True)
#         print("\n=== GENERATED (with cache) ===\n")
#         print(text2)
#     except Exception as e:
#         print("Cache generation failed (probably model.forward doesn't return per-block caches). Error:", e)
#         print("You can still use the simple generator.")


RuntimeError: MiniGPT class not available. Import it or copy the class into this file.